In [1]:
# Common imports and setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
import pickle
warnings.filterwarnings('ignore')
np.random.seed(126)


In [2]:
# Problem 1: Helper functions
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

def load_data_with_date(data_path, train_end, pred_start, pred_end):
    df = pd.read_csv(data_path)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)
    target_col = [c for c in df.columns if 'MW' in c.upper()][0]
    train_data = df[df.index <= pd.to_datetime(train_end)][target_col].values
    train_data = train_data[~np.isnan(train_data)]
    pred_actual = df[(df.index >= pd.to_datetime(pred_start)) & (df.index <= pd.to_datetime(pred_end))][target_col].values
    pred_actual = pred_actual[~np.isnan(pred_actual)]
    return train_data, pred_actual

def create_sequences(data, seq_len=96, pred_len=24):
    return np.array([data[i:i+seq_len] for i in range(len(data)-seq_len-pred_len+1)]), \
           np.array([data[i+seq_len:i+seq_len+pred_len] for i in range(len(data)-seq_len-pred_len+1)])

def smape_loss(y_true, y_pred):
    import tensorflow as tf
    eps = tf.keras.backend.epsilon()
    return tf.reduce_mean(2.0 * tf.abs(y_true - y_pred) / (tf.abs(y_true) + tf.abs(y_pred) + eps))

def build_model(seq_len, pred_len, improved=True):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    if improved:
        return Sequential([
            LSTM(128, return_sequences=True, input_shape=(seq_len, 1)), Dropout(0.3),
            LSTM(128, return_sequences=True), Dropout(0.3),
            LSTM(64, return_sequences=False), Dropout(0.2),
            Dense(64, activation='relu'), Dense(32, activation='relu'), Dense(pred_len)
        ])
    else:
        return Sequential([
            LSTM(64, return_sequences=True, input_shape=(seq_len, 1)), Dropout(0.2),
            LSTM(64, return_sequences=False), Dropout(0.2),
            Dense(32), Dense(pred_len)
        ])

def iterative_forecast(model, X_init, pred_hours, batch_size=24):
    predictions, current_seq = [], X_init.copy()
    for i in range(0, pred_hours, batch_size):
        pred_batch = model.predict(current_seq.reshape(1, current_seq.shape[0], 1), verbose=0)[0]
        predictions.append(pred_batch if i + batch_size <= pred_hours else pred_batch[:pred_hours - i])
        if i + batch_size <= pred_hours:
            current_seq = np.concatenate([current_seq[batch_size:], pred_batch])
    return np.concatenate(predictions)

def calculate_smape(y_true, y_pred):
    """Calculate sMAPE value"""
    eps = 1e-8
    return np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred) + eps))

def plot_model_architecture():
    try:
        from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
        fig, ax = plt.subplots(1, 1, figsize=(12, 7))
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 10)
        ax.axis('off')
        boxes = [('Raw Data', 1, 8), ('Normalization', 3.5, 8), ('Sequence Creation', 6, 8), ('LSTM Model', 3.5, 6),
                 ('Training', 1, 4), ('Iterative Forecast', 3.5, 4), ('Denormalization', 6, 4), ('Evaluation', 3.5, 2)]
        for text, x, y in boxes:
            ax.add_patch(FancyBboxPatch((x-0.75, y-0.4), 1.5, 0.8, boxstyle="round,pad=0.1",
                                       edgecolor='black', facecolor='lightblue', linewidth=2))
            ax.text(x, y, text, ha='center', va='center', fontsize=9, weight='bold')
        arrows = [(1, 8, 3.5, 8), (3.5, 8, 6, 8), (6, 7.6, 3.5, 6.4), (3.5, 5.6, 1, 4.4),
                  (1, 4, 3.5, 4), (3.5, 4, 6, 4), (6, 3.6, 3.5, 2.4)]
        for x1, y1, x2, y2 in arrows:
            ax.add_patch(FancyArrowPatch((x1, y1), (x2, y2), arrowstyle='->', mutation_scale=20,
                                       linewidth=2, color='darkblue'))
        ax.set_title('Neural Network Model Architecture', fontsize=14, weight='bold', pad=20)
        plt.tight_layout()
        plt.savefig('model_architecture.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("Model architecture diagram saved to model_architecture.png")
    except Exception as e:
        print(f"Error plotting model architecture: {e}")


In [9]:
# Problem 1: Separate training and prediction functions

def train_model(data_path, train_end, seq_len=96, problem_name='', retrain=False, epochs=100, batch_size=64):
    """
    Train the LSTM model for time series forecasting.
    
    Parameters:
    - data_path: path to the CSV file
    - train_end: end date for training data (e.g., '2017-12-31')
    - seq_len: sequence length for LSTM input
    - problem_name: name identifier for the problem (used for file naming)
    - retrain: if True, force retraining even if model exists
    - epochs: number of training epochs
    - batch_size: batch size for training
    
    Returns:
    - model: trained Keras model
    - scaler: fitted StandardScaler
    - train_scaled: scaled training data
    """
    print(f"\n{'='*60}")
    print(f"Training Model: {problem_name}")
    print(f"{'='*60}")
    
    # Load training data
    df = pd.read_csv(data_path)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)
    target_col = [c for c in df.columns if 'MW' in c.upper()][0]
    train_data = df[df.index <= pd.to_datetime(train_end)][target_col].values
    train_data = train_data[~np.isnan(train_data)]
    print(f"Training data: {len(train_data)} points")
    
    # Generate model and scaler file paths (use .weights.h5 for save_weights_only=True)
    model_name = 'model_problem_a.weights.h5' if 'A' in problem_name else 'model_problem_b.weights.h5'
    scaler_name = 'scaler_problem_a.pkl' if 'A' in problem_name else 'scaler_problem_b.pkl'
    
    # Prepare data
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_data.reshape(-1, 1)).flatten()
    pred_len = 24
    
    X_train, y_train = create_sequences(train_scaled, seq_len=seq_len, pred_len=pred_len)
    X_train_r = np.array(X_train, dtype=np.float32).reshape((len(X_train), seq_len, 1))
    y_train_all = np.array(y_train, dtype=np.float32)
    
    print(f"Training sequences: {len(X_train)}")
    
    try:
        from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
        
        # Check if saved model exists and not forcing retrain
        if not retrain and os.path.exists(model_name) and os.path.exists(scaler_name):
            print(f"Loading saved model from {model_name}...")
            model = build_model(seq_len, pred_len, improved=True)
            model.compile(optimizer='adam', loss=smape_loss, metrics=['mae'])
            model.load_weights(model_name)
            with open(scaler_name, 'rb') as f:
                scaler = pickle.load(f)
            train_scaled = scaler.transform(train_data.reshape(-1, 1)).flatten()
            print("Model and scaler loaded successfully!")
            return model, scaler, train_scaled
        
        # Train new model
        if retrain:
            print("Force retraining model...")
        else:
            print("Training new model...")
        
        model = build_model(seq_len, pred_len, improved=True)
        model.compile(optimizer='adam', loss=smape_loss, metrics=['mae'])
        
        checkpoint = ModelCheckpoint(model_name, monitor='loss', save_best_only=True, 
                                   save_weights_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1)
        
        print(f"Starting training for {epochs} epochs...")
        history = model.fit(X_train_r, y_train_all, epochs=epochs, batch_size=batch_size,
                           callbacks=[early_stopping, checkpoint], verbose=1)
        
        # Save scaler
        with open(scaler_name, 'wb') as f:
            pickle.dump(scaler, f)
        print(f"\n✓ Model saved to {model_name}")
        print(f"✓ Scaler saved to {scaler_name}")
        print(f"✓ Training completed!")
        
        return model, scaler, train_scaled
        
    except Exception as e:
        print(f"Error during training: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None


def predict_with_model(model, scaler, train_scaled, data_path, pred_start, pred_end, 
                      seq_len=96, problem_name=''):
    """
    Use trained model to make predictions.
    
    Parameters:
    - model: trained Keras model
    - scaler: fitted StandardScaler
    - train_scaled: scaled training data (last seq_len points will be used)
    - data_path: path to the CSV file
    - pred_start: start date for prediction
    - pred_end: end date for prediction
    - seq_len: sequence length (must match training)
    - problem_name: name identifier for the problem
    
    Returns:
    - smape: sMAPE metric value
    """
    print(f"\n{'='*60}")
    print(f"Making Predictions: {problem_name}")
    print(f"{'='*60}")
    
    if model is None or scaler is None:
        print("Error: Model or scaler is None. Please train the model first.")
        return None
    
    try:
        # Load actual values for comparison
        df = pd.read_csv(data_path)
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        df.set_index('Datetime', inplace=True)
        target_col = [c for c in df.columns if 'MW' in c.upper()][0]
        pred_actual = df[(df.index >= pd.to_datetime(pred_start)) & 
                         (df.index <= pd.to_datetime(pred_end))][target_col].values
        pred_actual = pred_actual[~np.isnan(pred_actual)]
        print(f"Prediction period: {len(pred_actual)} hours")
        
        # Make predictions
        print("Generating predictions...")
        long_term_pred = iterative_forecast(model, train_scaled[-seq_len:], len(pred_actual))[:len(pred_actual)]
        long_term_pred_actual = scaler.inverse_transform(long_term_pred.reshape(-1, 1)).flatten()
        pred_actual_vals = scaler.inverse_transform(pred_actual.reshape(-1, 1)).flatten()
        
        # Calculate sMAPE
        smape = calculate_smape(pred_actual_vals, long_term_pred_actual)
        print(f"\nForecasting Results - SMAPE: {smape:.4f}")
        
        # Plot results
        plot_len = min(2000, len(pred_actual_vals))
        time_idx = pd.date_range(start=pd.to_datetime(pred_start), periods=plot_len, freq='H')
        plt.figure(figsize=(16, 6))
        plt.plot(time_idx, pred_actual_vals[:plot_len], label='Actual', linewidth=1, alpha=0.7)
        plt.plot(time_idx, long_term_pred_actual[:plot_len], label='Predicted', linewidth=1, linestyle='--', alpha=0.7)
        plt.xlabel('Date')
        plt.ylabel('Energy Consumption (MW)')
        plt.title(f'{problem_name} (First {plot_len} Hours)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        filename = 'problem_a_forecast.png' if 'A' in problem_name else 'long_term_forecast.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ Plot saved to {filename}")
        
        return smape
        
    except Exception as e:
        print(f"Error during prediction: {e}")
        import traceback
        traceback.print_exc()
        return None


def forecast(data_path, train_end, pred_start, pred_end, seq_len=96, problem_name='', 
             retrain=False, epochs=100, batch_size=64):
    """
    Complete workflow: train model and make predictions.
    This function combines train_model and predict_with_model for convenience.
    """
    # Step 1: Train model
    model, scaler, train_scaled = train_model(data_path, train_end, seq_len, problem_name, retrain, epochs, batch_size)
    
    if model is None:
        return None
    
    # Step 2: Make predictions
    smape = predict_with_model(model, scaler, train_scaled, data_path, pred_start, pred_end, 
                              seq_len, problem_name)
    
    return smape


In [ ]:
# Plot model architecture
plot_model_architecture()

# Problem 1A: Step 1 - Train the model
print("="*60)
print("Problem 1A: Training Model")
print("="*60)
model_a, scaler_a, train_scaled_a = train_model(
    'EnergyConsumption_hourly.csv', 
    '2017-12-31', 
    seq_len=96, 
    problem_name='Problem A: Forecasting (2018-01 to 2018-08)', 
    retrain=False,
    epochs=100,
    batch_size=64
)


Model architecture diagram saved to model_architecture.png
Problem 1A: Training Model

Training Model: Problem A: Forecasting (2018-01 to 2018-08)
Training data: 116116 points
Training sequences: 115997
Training new model...
Starting training for 100 epochs...
Epoch 1/100


In [ ]:
# Problem 1A: Step 2 - Make predictions after training completes
if model_a is not None:
    smape_a = predict_with_model(
        model_a, scaler_a, train_scaled_a,
        'EnergyConsumption_hourly.csv',
        '2018-01-01', '2018-08-31',
        seq_len=96,
        problem_name='Problem A: Forecasting (2018-01 to 2018-08)'
    )
else:
    print("Training failed, cannot make predictions.")
    smape_a = None


In [ ]:
# Problem 1B: Step 1 - Train the model
print("="*60)
print("Problem 1B: Training Model")
print("="*60)
model_b, scaler_b, train_scaled_b = train_model(
    'EnergyConsumption_hourly.csv', 
    '2016-12-31', 
    seq_len=168, 
    problem_name='Problem B: Long-Term Forecasting (2017-01 to 2018-08)', 
    retrain=False,
    epochs=100,
    batch_size=64
)


In [ ]:
# Problem 1B: Step 2 - Make predictions after training completes
if model_b is not None:
    smape_b = predict_with_model(
        model_b, scaler_b, train_scaled_b,
        'EnergyConsumption_hourly.csv',
        '2017-01-01', '2018-08-31',
        seq_len=168,
        problem_name='Problem B: Long-Term Forecasting (2017-01 to 2018-08)'
    )
else:
    print("Training failed, cannot make predictions.")
    smape_b = None


In [ ]:
# Problem 1: Summary
print("\n" + "="*60 + "\nSummary:")
if smape_a is not None:
    print(f"  Problem A (2018-01 to 2018-08) - SMAPE: {smape_a:.4f}")
if smape_b is not None:
    print(f"  Problem B (2017-01 to 2018-08) - SMAPE: {smape_b:.4f}")
print("="*60)


In [ ]:
# Problem 2: Imports
import random
import math
from matplotlib import colors
random.seed(126)


In [ ]:
# Problem 2: Ising Model Simulation Functions
def initialize_grid(dim):
    return np.random.choice([-1, 1], size=(dim, dim))

def energy_change(grid, i, j):
    n = grid.shape[0]
    left, right = (i - 1) % n, (i + 1) % n
    up, down = (j + 1) % n, (j - 1) % n
    return 2 * grid[i, j] * (grid[left, j] + grid[right, j] + grid[i, up] + grid[i, down])

def spin_flip(grid, T):
    n = grid.shape[0]
    i, j = random.randint(0, n - 1), random.randint(0, n - 1)
    delta_E = energy_change(grid, i, j)
    if delta_E < 0 or random.random() < math.exp(-delta_E / T):
        grid[i, j] = -grid[i, j]
    return grid

def ising_simulation(n, T, steps=100):
    """Simulate 2D Ising model using Metropolis algorithm"""
    grid = initialize_grid(n)
    for step in range(steps):
        for _ in range(n * n):
            grid = spin_flip(grid, T)
    return grid

def generate_data(size, num_temp, temp_min=1.0, temp_max=3.5, repeat=1, max_iter=None):
    """Generate training/test data from Ising model simulations"""
    if max_iter is None:
        max_iter = size**2
    X = np.zeros((num_temp * repeat, size**2))
    y_label = np.zeros((num_temp * repeat, 1))
    y_temp = np.zeros((num_temp * repeat, 1))
    temps = np.linspace(temp_min, temp_max, num=num_temp)
    for i in range(repeat):
        for j in range(num_temp):
            grid = ising_simulation(size, temps[j], max_iter)
            X[i*num_temp + j, :] = grid.reshape(1, grid.size)
            y_label[i*num_temp + j, :] = (temps[j] > 2.269)
            y_temp[i*num_temp + j, :] = temps[j]
            print(f"Generated {i*num_temp + j + 1}/{num_temp * repeat}", end='\r')
    print()
    return X, y_label, y_temp


In [ ]:
# Problem 2: Model Building Functions
def build_ising_model(input_dim, task_type='classification'):
    """Build neural network model (TensorFlow or sklearn)"""
    try:
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense, Dropout
        from tensorflow.keras.optimizers import Adam
        model = Sequential([
            Dense(128, activation='relu', input_shape=(input_dim,)),
            Dropout(0.3), Dense(64, activation='relu'),
            Dropout(0.2), Dense(32, activation='relu'),
            Dense(1, activation='sigmoid' if task_type == 'classification' else None)
        ])
        model.compile(optimizer=Adam(0.001),
                     loss='binary_crossentropy' if task_type == 'classification' else 'mean_absolute_error',
                     metrics=['accuracy' if task_type == 'classification' else 'mae'])
        return model, 'keras'
    except ImportError:
        from sklearn.neural_network import MLPClassifier, MLPRegressor
        if task_type == 'classification':
            return MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=500, 
                                random_state=126, early_stopping=True), 'sklearn'
        else:
            return MLPRegressor(hidden_layer_sizes=(128, 64, 32), max_iter=500,
                              random_state=126, early_stopping=True), 'sklearn'

def plot_model_flowchart(task_name, model_type, save_path):
    """Create model flowchart"""
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axis('off')
    layers = ['Input\n(625)', 'Dense\n(128)', 'Dense\n(64)', 'Dense\n(32)', 
              'Output\n(1)' if model_type == 'regression' else 'Output\n(0/1)']
    y_pos = np.linspace(0.9, 0.1, len(layers))
    for i, (layer, y) in enumerate(zip(layers, y_pos)):
        rect = plt.Rectangle((0.4 - 0.08, y - 0.04), 0.16, 0.08,
                           facecolor='lightblue', edgecolor='black', linewidth=2)
        ax.add_patch(rect)
        ax.text(0.4, y, layer, ha='center', va='center', fontsize=9, fontweight='bold')
        if i < len(layers) - 1:
            ax.arrow(0.4, y - 0.04, 0, -(y_pos[i] - y_pos[i+1] - 0.08),
                    head_width=0.015, head_length=0.015, fc='black', ec='black')
    ax.text(0.5, 0.95, f'{task_name} Model Flowchart', ha='center', fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def train_and_evaluate(model, model_type, X_train, y_train, X_test, y_test, task_name):
    """Train model and return predictions"""
    if model_type == 'keras':
        from tensorflow.keras.callbacks import EarlyStopping
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2,
                 callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
                 verbose=0)
        y_pred = model.predict(X_test, verbose=0)
        if task_name == 'classification':
            y_pred = (y_pred > 0.5).astype(int).flatten()
        else:
            y_pred = y_pred.flatten()
    else:
        model.fit(X_train, y_train.flatten())
        y_pred = model.predict(X_test)
    return y_pred

def plot_results(y_true, y_pred, y_temp_test, task_name, metric_name, save_path):
    """Plot results vs temperature"""
    from sklearn.metrics import accuracy_score, mean_absolute_error
    unique_temps = np.unique(y_temp_test)
    metrics_by_temp, temps_list = [], []
    for temp in unique_temps:
        mask = (y_temp_test.flatten() == temp)
        if np.sum(mask) > 0:
            if task_name == 'classification':
                metric = accuracy_score(y_true[mask], y_pred[mask])
            else:
                metric = mean_absolute_error(y_true[mask], y_pred[mask])
            metrics_by_temp.append(metric)
            temps_list.append(temp)
    plt.figure(figsize=(10, 6))
    plt.plot(temps_list, metrics_by_temp, 'o-', linewidth=2, markersize=8,
            color='blue' if task_name == 'classification' else 'green')
    plt.axvline(x=2.269, color='r', linestyle='--', linewidth=2, label='Tc = 2.269')
    plt.xlabel('Temperature (T)', fontsize=12)
    plt.ylabel(metric_name, fontsize=12)
    plt.title(f'Task {"A" if task_name == "classification" else "B"}: {metric_name} vs Temperature', 
             fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=11)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


In [ ]:
# Problem 2: Generate training and test data
print("=" * 60)
print("Problem 2: Ising Model - Neural Network Tasks")
print("=" * 60)
print("\nGenerating training data...")
X_train, y_label_train, y_temp_train = generate_data(
    size=25, num_temp=51, temp_min=1.0, temp_max=3.5, repeat=20, max_iter=625)
print("\nGenerating test data...")
X_test, y_label_test, y_temp_test = generate_data(
    size=25, num_temp=21, temp_min=1.0, temp_max=3.5, repeat=20, max_iter=625)
print(f"\nTraining: {X_train.shape[0]} samples, Test: {X_test.shape[0]} samples")


In [ ]:
# Task A: Classification
print("\n" + "=" * 60)
print("Task A: Classification")
print("=" * 60)
model, model_type = build_ising_model(X_train.shape[1], 'classification')
plot_model_flowchart("Task A", "classification", "task_a_model_flowchart.png")
print("Training model...")
y_pred = train_and_evaluate(model, model_type, X_train, y_label_train, X_test, y_label_test, 'classification')
y_true = y_label_test.flatten().astype(int)
from sklearn.metrics import accuracy_score
print(f"Overall Test Accuracy: {accuracy_score(y_true, y_pred):.4f}")
plot_results(y_true, y_pred, y_temp_test, 'classification', 'Test Accuracy', 'task_a_accuracy_vs_temp.png')
print("Plot saved to task_a_accuracy_vs_temp.png")
print("Observations: Accuracy decreases near critical temperature (Tc = 2.269)")


In [ ]:
# Task B: Regression
print("\n" + "=" * 60)
print("Task B: Regression")
print("=" * 60)
model, model_type = build_ising_model(X_train.shape[1], 'regression')
plot_model_flowchart("Task B", "regression", "task_b_model_flowchart.png")
print("Training model...")
y_pred = train_and_evaluate(model, model_type, X_train, y_temp_train, X_test, y_temp_test, 'regression')
y_true = y_temp_test.flatten()
from sklearn.metrics import mean_absolute_error
print(f"Overall Test MAE: {mean_absolute_error(y_true, y_pred):.4f}")
plot_results(y_true, y_pred, y_temp_test, 'regression', 'Test MAE', 'task_b_mae_vs_temp.png')
print("Plot saved to task_b_mae_vs_temp.png")
print("Observations: MAE increases near critical temperature (Tc = 2.269)")


In [ ]:
# Problem 2: Summary
print("\n" + "=" * 60)
print("All tasks completed!")
print("=" * 60)
